In [6]:
import pandas as pd
import folium
from folium.plugins import Search, MarkerCluster, LocateControl
import os

# Load API key from environment variable (or you can hardcode it for testing purposes)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", "AlzaSyCcUskQaX1cPWgiZCKRqhFfYkevqbYtLyE")

# Load the CSV data into a pandas DataFrame
data = pd.read_csv("data1.csv")

# Initialize a folium map centered on a default location (adjust based on your data)
mymap = folium.Map(location=[data['latitude'].mean(), data['longitude'].mean()], zoom_start=10)

# Create a MarkerCluster to group the markers
marker_cluster = MarkerCluster().add_to(mymap)

# Loop through the data to add markers to the map
for index, row in data.iterrows():
    # Create a popup message for each location
    popup_message = f"<strong>{row['name']}</strong><br>{row['location']}<br>{row['type']}"
    
    # Create a marker
    marker = folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=popup_message,
        icon=folium.Icon(color='red', icon='info-sign')  # Change icon as needed
    )
    
    # Add marker to the cluster
    marker.add_to(marker_cluster)
    
    # Add a label using DivIcon (displaying the location name)
    label = folium.map.Marker(
        [row['latitude'], row['longitude']],
        icon=folium.DivIcon(
            html=f'<div style="font-size: 12px; color: black;">{row["name"]}</div>'  # Customize font and color here
        )
    )
    label.add_to(mymap)

# Add a search bar to search by 'location' field in the CSV data
search = Search(
    layer=marker_cluster,  # Search within the MarkerCluster layer
    search_label='location',  # Search by location in the CSV data
    placeholder='Search for a city or location',
    collapsed=False,  # Keep the search bar expanded
).add_to(mymap)

# Add a layer toggle for different map types (base layers)
folium.TileLayer('Stamen Watercolor').add_to(mymap)
folium.TileLayer('CartoDB positron').add_to(mymap)

# Add a Street View-like layer (Google Street View layer using external tile URL)
folium.TileLayer(
    tiles=f'https://mt1.google.com/vt/lyrs=y&x={{x}}&y={{y}}&z={{z}}&key={GOOGLE_API_KEY}',
    attr='Google Street View',
    name='Street View',
    overlay=False,
    control=True
).add_to(mymap)

# Add the layer control to toggle between map layers
folium.LayerControl().add_to(mymap)

# Add real-time location control (LocateControl)
LocateControl().add_to(mymap)

# Save the map as an HTML file
mymap.save("interactive_map.html")

# Display the map in a Jupyter notebook or script (if you're using Jupyter)
mymap
